<h1 align=center><font size = 5> <font color='#0489B1'>Modelo de Regresion con Keras</font></h1>


## Introducción


Keras es una API de alto nivel para construir modelos de deep learning.

En este laboratorio aprenderemos a construir un modelo de regresión.


<h2>Modelo de regresión con Keras</h2>

<h3>Objetivos<h3>    
<h5> 1. Usar Keras para construir un modelo de regresión</h5>
<h5> 2. Descargar y limpiar un dataset </h5>
<h5> 3. Construir una red neuronal </h5>
<h5> 4. Entrenar y testear la red </h5>     


## Tabla de contenido

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
1. <a href="#item31">Descargar y limpiar un dataset</a>  
2. <a href="#item32">Importar Keras</a>  
3. <a href="#item33">Construir una red neuroanl</a>  
4. <a href="#item34">Entrenar y testear una red neuronal</a>  

</font>
</div>


<a id="item31"></a>


## Download and Clean Dataset


Importemos Pandas y NumPy.


In [23]:
import pandas as pd
import numpy as np


<strong> El dataset trata acerca de la resistencia (strength) de diferentes muestras de concreto basadas en el volumen de diferentes ingredientes que se utilizan para preparalo. Los ingredientes incluyen:
</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>


Descarguemos los datos y leámoslos en un dataframe.


In [24]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


#### Veamos cuántos datos tenemos


In [25]:
concrete_data.shape

(1030, 9)

Hay 1030 muestras para entrenar nuestro  modelo. Dado que son pocas muestras, debemos tener especial cuidado en no sobreajustar los datos.



Comprobemos si hay valores faltantes:


In [26]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [27]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

Los datos lucen limpios y prontos para ser utilizados en la construcción del modelo.

#### Dividimos los datos en predictores y target (objetivo)


La variable objetivo en este problema es strength; los predictores son las columnas restantes.


In [28]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

<a id="item2"></a>


Sanity check:


In [29]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [30]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

El último paso es normalizar los datos restando la media y dividiendo por la desviación estándar.


In [31]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Guardemos el número de predictores en _n_cols_ ya que necesitaremos este número para construir la red.


In [32]:
n_cols = predictors_norm.shape[1] # number of predictors
n_cols

8

<a id="item1"></a>


<a id='item32'></a>


## Importando Keras


In [33]:
import keras

Importemos el resto de los paquetes de la librería de Keras que necesitaremos para construir nuestro modelo de regresión.


In [34]:
from keras.models import Sequential
from keras.layers import Dense

<a id='item33'></a>


## Construyendo una red neuronal


Crearemos una función que define nuestro modelo de regresión para llamarla cuando necesitemos crear el modelo.



In [35]:
# definimos el modelo de regresión
def regression_model():
    # creamos el modelo
    model = Sequential()
    model.add(Dense(50, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    
    # compilamos el modelo
    model.compile(optimizer='adam', loss='mean_squared_error',metrics=['mse'])
    return model

La función de arriba crea un modelo que tiene 2 capas ocultas, cada una de 50 unidades.

<a id="item4"></a>


<a id='item34'></a>


## Entrenando y teseteando la red


Llamemos a la función para crear nuestro modelo.


In [36]:
# construimos el modelo
model = regression_model()

Entrenamos y testeamos el modelo al mismo tiempo usando el método _fit_. Dejaremos afuera el 30% de los datos para validación y entrenaremos el modelo para 100 epochs.


In [37]:
# ajustamos el modelo
m = model.fit(predictors_norm, target, validation_split=0.3, epochs=100, verbose=2)

Epoch 1/100
23/23 - 0s - loss: 1645.0375 - mse: 1645.0375 - val_loss: 1140.0239 - val_mse: 1140.0239
Epoch 2/100
23/23 - 0s - loss: 1497.7294 - mse: 1497.7294 - val_loss: 1002.9310 - val_mse: 1002.9310
Epoch 3/100
23/23 - 0s - loss: 1258.8268 - mse: 1258.8268 - val_loss: 796.2897 - val_mse: 796.2897
Epoch 4/100
23/23 - 0s - loss: 906.9604 - mse: 906.9604 - val_loss: 548.8319 - val_mse: 548.8319
Epoch 5/100
23/23 - 0s - loss: 541.5153 - mse: 541.5153 - val_loss: 337.9198 - val_mse: 337.9198
Epoch 6/100
23/23 - 0s - loss: 314.7509 - mse: 314.7509 - val_loss: 244.4211 - val_mse: 244.4211
Epoch 7/100
23/23 - 0s - loss: 246.3289 - mse: 246.3289 - val_loss: 209.5467 - val_mse: 209.5467
Epoch 8/100
23/23 - 0s - loss: 224.8444 - mse: 224.8444 - val_loss: 196.8851 - val_mse: 196.8851
Epoch 9/100
23/23 - 0s - loss: 210.1174 - mse: 210.1174 - val_loss: 189.5222 - val_mse: 189.5222
Epoch 10/100
23/23 - 0s - loss: 199.7482 - mse: 199.7482 - val_loss: 181.9310 - val_mse: 181.9310
Epoch 11/100
23/23 

Epoch 86/100
23/23 - 0s - loss: 41.4634 - mse: 41.4634 - val_loss: 116.9966 - val_mse: 116.9966
Epoch 87/100
23/23 - 0s - loss: 41.2495 - mse: 41.2495 - val_loss: 126.6615 - val_mse: 126.6615
Epoch 88/100
23/23 - 0s - loss: 40.2516 - mse: 40.2516 - val_loss: 121.6855 - val_mse: 121.6855
Epoch 89/100
23/23 - 0s - loss: 39.4056 - mse: 39.4056 - val_loss: 122.2474 - val_mse: 122.2474
Epoch 90/100
23/23 - 0s - loss: 38.6336 - mse: 38.6336 - val_loss: 118.3280 - val_mse: 118.3280
Epoch 91/100
23/23 - 0s - loss: 38.4374 - mse: 38.4374 - val_loss: 118.3810 - val_mse: 118.3810
Epoch 92/100
23/23 - 0s - loss: 37.9610 - mse: 37.9610 - val_loss: 137.6325 - val_mse: 137.6325
Epoch 93/100
23/23 - 0s - loss: 37.7209 - mse: 37.7209 - val_loss: 127.5167 - val_mse: 127.5167
Epoch 94/100
23/23 - 0s - loss: 36.9463 - mse: 36.9463 - val_loss: 120.8960 - val_mse: 120.8960
Epoch 95/100
23/23 - 0s - loss: 36.1867 - mse: 36.1867 - val_loss: 138.9015 - val_mse: 138.9015
Epoch 96/100
23/23 - 0s - loss: 35.6289 

<strong>En este link [link](https://keras.io/models/sequential?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DL0101EN-SkillsNetwork-20718188&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DL0101EN-SkillsNetwork-20718188&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DL0101EN-SkillsNetwork-20718188&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ) puede aprender acerca de otras funciones que puede usar para predicción o evaluación.</strong>


Puede variar los siguientes parámetros para mejorar la preformance:

1. El número de neuronas en las capas ocultas.
2. El número de capas ocultas.
3. El número de epochs.


In [39]:
m.history['val_mse']
#https://machinelearningmastery.com/custom-metrics-deep-learning-keras-python/

[1140.02392578125,
 1002.9310302734375,
 796.2897338867188,
 548.8319091796875,
 337.9198303222656,
 244.421142578125,
 209.54672241210938,
 196.8850860595703,
 189.5221710205078,
 181.93096923828125,
 179.76934814453125,
 174.8551788330078,
 172.8262176513672,
 170.7350616455078,
 167.91444396972656,
 166.42063903808594,
 163.84686279296875,
 162.10140991210938,
 161.72962951660156,
 160.74169921875,
 161.1456756591797,
 160.34310913085938,
 160.5050048828125,
 159.61431884765625,
 161.5148468017578,
 162.41058349609375,
 162.47320556640625,
 161.0438232421875,
 161.7865753173828,
 161.79019165039062,
 161.9120330810547,
 164.6956787109375,
 162.9900665283203,
 158.6812286376953,
 163.6772003173828,
 162.3165740966797,
 162.44390869140625,
 160.0847625732422,
 159.56442260742188,
 159.0526885986328,
 159.63328552246094,
 159.32933044433594,
 157.16543579101562,
 159.65365600585938,
 160.48504638671875,
 156.1938018798828,
 155.4173583984375,
 155.128662109375,
 153.7607421875,
 153.46

In [19]:
m[0]

TypeError: 'History' object is not subscriptable